In [ ]:
# Python Built-Ins:
import os
from typing import Optional
import sys
import json
import time

# External Dependencies:
import boto3
from botocore.config import Config
import botocore

def get_bedrock_client(
    runtime: Optional[bool] = True,
    aws_session_token: Optional[str] = None
):
    if runtime:
        service_name = 'bedrock-runtime'
    else:
        service_name = 'bedrock'

    # Using AWS CLI's configured credentials
    bedrock_runtime = boto3.client(
        service_name=service_name,
        region_name="us-west-2",  # Change to your preferred region
        aws_session_token=aws_session_token  # Optional
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_runtime._endpoint)
    return bedrock_runtime

In [ ]:
bedrock_runtime = get_bedrock_client()

Fetch daily gold prices for the last 180 days with rate limiting

In [1]:
import requests
import boto3
import json
from datetime import datetime, timedelta
import time

# Configuration
API_KEY = "1PA3azCLG7VJkFaCoHnN9g==lAud9H2ACUxiNlMF"  # Note: This appears to be a real API key - consider using environment variables
COMMODITY = "gold"
REGION = "us-west-2"  # AWS region for Bedrock

def fetch_180day_gold_prices():
    """Fetch daily gold prices for the last 180 days with rate limiting"""
    prices = []
    failed_attempts = 0
    max_attempts = 5
    
    for i in range(180):
        date = (datetime.now() - timedelta(days=i)).strftime('%Y-%m-%d')
        api_url = f'https://api.api-ninjas.com/v1/commodityprice?name={COMMODITY}&date={date}'
        
        try:
            response = requests.get(api_url, headers={'X-Api-Key': API_KEY}, timeout=10)
            
            if response.status_code == 200:
                data = response.json()
                if data.get('price'):
                    prices.append({
                        'date': date,
                        'price': float(data['price']),  # Ensure numeric value
                        'unit': data.get('unit', 'USD/oz')
                    })
                else:
                    print(f"Warning: No price data for {date}")
            else:
                print(f"Warning: Failed to fetch {date} - {response.status_code}")
                failed_attempts += 1
                if failed_attempts >= max_attempts:
                    raise Exception("Too many consecutive API failures")
                
                # Exponential backoff for rate limiting
                time.sleep(min(2 ** failed_attempts, 60))
                
        except requests.exceptions.RequestException as e:
            print(f"Network error for {date}: {str(e)}")
            time.sleep(5)
            continue
    
    return sorted(prices, key=lambda x: x['date'])

def analyze_trend_with_llm(prices):
    """Analyze price trends using Claude 3 with enhanced prompt"""
    bedrock = boto3.client('bedrock-runtime', region_name=REGION)
    
    # Create summary statistics for the LLM
    price_values = [p['price'] for p in prices if p['price']]
    if not price_values:
        return "Error: No valid price data to analyze"
    
    min_price = min(price_values)
    max_price = max(price_values)
    last_price = price_values[-1]
    
    prompt = f"""Human: Analyze this 180-day gold price trend (USD/oz):
    Period: {prices[0]['date']} to {prices[-1]['date']}
    Data Points: {len(prices)} days
    Price Range: ${min_price:.2f} to ${max_price:.2f}
    Current Price: ${last_price:.2f}
    
    Please provide:
    1. **Trend Analysis**:
       - Overall direction and magnitude
       - Key support/resistance levels
       - Volatility patterns
    
    2. **Market Context**:
       - 3 macroeconomic factors influencing this trend
       - Correlation with typical market indicators
    
    3. **Trading Insights**:
       - Risk assessment for buyers/sellers
       - 2 potential entry/exit strategies
       - Long-term outlook considerations
    
    Format your response with clear markdown headings and bullet points.
    Include specific price references when possible.
    Assistant:
    """
    
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1500,  # Increased for more detailed analysis
        "messages": [{"role": "user", "content": prompt}]
    }
    
    try:
        response = bedrock.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            body=json.dumps(body)
        )
        return json.loads(response['body'].read())['content'][0]['text']
    except Exception as e:
        return f"LLM Analysis Error: {str(e)}"

if __name__ == "__main__":
    try:
        print("Fetching 180 days of gold prices...")
        gold_prices = fetch_180day_gold_prices()
        
        print(f"\nSuccessfully fetched {len(gold_prices)} data points")
        print("Date Range:", gold_prices[0]['date'], "to", gold_prices[-1]['date'])
        print("\nSample Data:")
        for day in gold_prices[:5] + gold_prices[-5:]:
            print(f"{day['date']}: {day['price']:.2f} {day['unit']}")
        
        print("\nAnalyzing trends with LLM...")
        analysis = analyze_trend_with_llm(gold_prices)
        print("\nGold Market Analysis (180 Days):")
        print(analysis)
        
    except Exception as e:
        print(f"Fatal Error: {str(e)}")


Fetching 180 days of gold prices...

Successfully fetched 180 data points
Date Range: 2024-10-13 to 2025-04-10

Sample Data:
2024-10-13: 3231.10 USD/oz
2024-10-14: 3231.10 USD/oz
2024-10-15: 3231.10 USD/oz
2024-10-16: 3231.10 USD/oz
2024-10-17: 3231.10 USD/oz
2025-04-06: 3232.60 USD/oz
2025-04-07: 3232.60 USD/oz
2025-04-08: 3232.60 USD/oz
2025-04-09: 3232.60 USD/oz
2025-04-10: 3232.60 USD/oz

Analyzing trends with LLM...

Gold Market Analysis (180 Days):
## Trend Analysis

- **Overall direction and magnitude**:
  - The gold price trend exhibits a marginally bullish direction, with a modest upward movement from $3231.10 to $3232.60 over the 180-day period.
  - The magnitude of the price change is relatively small, with a range of only $1.50.

- **Key support/resistance levels**:
  - Support level: $3231.10 (the lowest price observed during this period)
  - Resistance level: $3232.60 (the current price and the highest price observed)

- **Volatility patterns**:
  - The price trend displa